In [4]:
from loadFilteredData import *
#loading all data
origImg, filimg = loadAllTopicData()

['abbey', 'airport_terminal', 'amphitheater', 'amusement_park', 'aquarium', 'aqueduct', 'art_gallery', 'assembly_line', 'auditorium']
Loading Topic1: abbey
Loading Topic1: airport_terminal
Loading Topic1: amphitheater
Loading Topic1: amusement_park
Loading Topic1: aquarium
Loading Topic1: aqueduct
Loading Topic1: art_gallery
Loading Topic1: assembly_line
Loading Topic1: auditorium
(9000, 128, 128, 3)
(9000, 128, 128, 3)


In [17]:
def threeChannelHistogram(images, bins):
    histograms = []
    for image in images:
        redHist = np.histogram(image[:, :, 0], bins=bins, range=(0, 255))[0]
        greenHist = np.histogram(image[:, :, 0], bins=bins, range=(0, 255))[0]
        blueHist = np.histogram(image[:, :, 0], bins=bins, range=(0, 255))[0]
        #print(np.hstack((redHist, greenHist, blueHist)))
        histograms.append(np.hstack((redHist, greenHist, blueHist)))
        #histograms.append(np.histogram(image, bins=bins, range=(0, 255))[0])
    return np.stack(histograms)
threeChannelHistogram([origImg[0]], 50).shape


(1, 150)

In [5]:
from loadFilteredData import *
X, y, Xtest, ytest = createData([origImg, filimg], .9)

9000
9000


In [4]:
X.shape

(16200, 128, 128, 3)

In [5]:
Xtest.shape

(1800, 128, 128, 3)

In [6]:
np.sum(ytest, axis=0)

array([900., 900.])

In [7]:
from preprocessing import *
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D, Dropout
from keras.layers import Activation, Dropout, Flatten, Dense
# create model

Using TensorFlow backend.


In [8]:
input_shape = (128, 128, 3)
num_classes = 2

In [15]:
model = Sequential()
model.add(Dense(64, input_shape=input_shape))
model.add(Flatten())
model.add(Activation('relu'))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dense(2))
model.add(Activation('softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

In [ ]:
model.fit(X, y, epochs=150, batch_size=64)

Epoch 1/150


In [24]:
def DnCNN(depth,filters=64,image_channels=1, use_bnorm=True):
    layer_count = 0
    inpt = Input(shape=(None,None,image_channels),name = 'input'+str(layer_count))
    # 1st layer, Conv+relu
    layer_count += 1
    x = Conv2D(filters=filters, kernel_size=(3,3), strides=(1,1),kernel_initializer='Orthogonal', padding='same',name = 'conv'+str(layer_count))(inpt)
    layer_count += 1
    x = Activation('relu',name = 'relu'+str(layer_count))(x)
    # depth-2 layers, Conv+BN+relu
    for i in range(depth-2):
        layer_count += 1
        x = Conv2D(filters=filters, kernel_size=(3,3), strides=(1,1),kernel_initializer='Orthogonal', padding='same',use_bias = False,name = 'conv'+str(layer_count))(x)
        if use_bnorm:
            layer_count += 1
            #x = BatchNormalization(axis=3, momentum=0.1,epsilon=0.0001, name = 'bn'+str(layer_count))(x) 
        x = BatchNormalization(axis=3, momentum=0.0,epsilon=0.0001, name = 'bn'+str(layer_count))(x)
        layer_count += 1
        x = Activation('relu',name = 'relu'+str(layer_count))(x)  
    # last layer, Conv
    layer_count += 1
    x = Conv2D(filters=image_channels, kernel_size=(3,3), strides=(1,1), kernel_initializer='Orthogonal',padding='same',use_bias = False,name = 'conv'+str(layer_count))(x)
    layer_count += 1
    x = Subtract(name = 'subtract' + str(layer_count))([inpt, x])   # input - noise
    model = Model(inputs=inpt, outputs=x)
    
    return model

# define loss
def sum_squared_error(y_true, y_pred):
    #return K.mean(K.square(y_pred - y_true), axis=-1)
    #return K.sum(K.square(y_pred - y_true), axis=-1)/2
    return K.sum(K.square(y_pred - y_true))/2

In [26]:
model = DnCNN(depth=17,filters=64,image_channels=3,use_bnorm=True)
model.compile(optimizer=Adam(0.001), loss=sum_squared_error)

In [27]:
model.fit(origImg, filimg, steps_per_epoch=2000,epochs=5)

Epoch 1/5


InternalError: Dst tensor is not initialized.
	 [[Node: _recv_input0_1_0/_1077 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/gpu:0", send_device="/job:localhost/replica:0/task:0/cpu:0", send_device_incarnation=1, tensor_name="edge_632__recv_input0_1_0", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/gpu:0"]()]]

In [8]:
filimg.shape

(9000, 128, 128, 3)